In [53]:
from src.constants import places
import requests
import pprint
import re
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import os

In [4]:
from pymongo import MongoClient

client = MongoClient()

def connectCollection(database, collection):
    ''' return collection from db crunchbase'''
    db = client[database]
    coll = db[collection]
    return db, coll

In [5]:
db, coll = connectCollection('dbcompanies','companies_clean')

In [16]:
companies = db.companies_clean.find({ 
    "money_raised":{
        "$gte": 1000000
    }
}
) 

In [17]:
data_companies = pd.DataFrame(companies)

In [49]:
geo = []
for e in range(len(data_companies)):
    geo.append(data_companies.coordinates[e]['coordinates']) 

In [50]:
geo

[[-122.323895, 37.566879],
 [-111.9035, 33.8171],
 [-122.161523, 37.446823],
 [-122.3254705, 37.5645375],
 [-73.985506, 40.757929],
 [-118.243425, 34.052187],
 [-71.24762, 42.275263],
 [-118.4054182, 34.0698494],
 [-113.515769, 53.544711],
 [-82.702449, 27.913011],
 [18.036587, 59.337864],
 [18.036587, 59.337864],
 [-82.702449, 27.913011],
 [-86.0547016, 39.9052263],
 [-122.397582, 37.785271],
 [-121.949417, 37.317735],
 [-78.8267112, 35.8535092],
 [-0.1020265, 51.5196135],
 [-72.5196907, 42.3756409],
 [-122.0780632, 37.5561712],
 [-73.987764, 40.744618],
 [77.594376, 12.971606],
 [-71.1469003, 42.6751888],
 [-0.0936688, 51.5119418],
 [-73.9822211, 40.7454098],
 [-122.0884286, 37.4189073],
 [-122.1156934, 37.380653],
 [-74.0136605, 40.7078343],
 [-73.994797, 40.727216],
 [-118.489748, 34.007112],
 [-122.3399782, 47.6103008],
 [-71.101375, 42.364014],
 [-96.0791435, 41.2601663],
 [-122.3764714, 37.7405936],
 [-73.990967, 40.73343],
 [-74.011114, 40.707954],
 [-80.001933, 40.438423],
 [-

In [60]:
geo_point = []
for loc in range(len(data_companies)):
    geo_point.append(data_companies.coordinates[loc])  

In [63]:
geo_point

[{'type': 'Point', 'coordinates': [-122.323895, 37.566879]},
 {'type': 'Point', 'coordinates': [-111.9035, 33.8171]},
 {'type': 'Point', 'coordinates': [-122.161523, 37.446823]},
 {'type': 'Point', 'coordinates': [-122.3254705, 37.5645375]},
 {'type': 'Point', 'coordinates': [-73.985506, 40.757929]},
 {'type': 'Point', 'coordinates': [-118.243425, 34.052187]},
 {'type': 'Point', 'coordinates': [-71.24762, 42.275263]},
 {'type': 'Point', 'coordinates': [-118.4054182, 34.0698494]},
 {'type': 'Point', 'coordinates': [-113.515769, 53.544711]},
 {'type': 'Point', 'coordinates': [-82.702449, 27.913011]},
 {'type': 'Point', 'coordinates': [18.036587, 59.337864]},
 {'type': 'Point', 'coordinates': [18.036587, 59.337864]},
 {'type': 'Point', 'coordinates': [-82.702449, 27.913011]},
 {'type': 'Point', 'coordinates': [-86.0547016, 39.9052263]},
 {'type': 'Point', 'coordinates': [-122.397582, 37.785271]},
 {'type': 'Point', 'coordinates': [-121.949417, 37.317735]},
 {'type': 'Point', 'coordinates'

In [69]:
def near(geo_point, radio_meters):
        geopoint = geo_point
        return list(db.companies_clean.find({
        "coordinates.coordinates": {
         "$near": {
             "$geometry": geopoint,
             "$maxDistance": radio_meters
         }}})) 

radio_meters = 5000
list_number_offices = []
list_offices=[]
for i in range(len(data_companies)):
    num_offices =  near(geo_point[i], radio_meters)
    list_offices.append(num_offices)
    list_number_offices.append(len(num_offices))
data_companies['number of offices near'] = list_number_offices

In [70]:
data_companies

,_id,name,employees,year,category,id,money_raised,city,country,latitude,longitude,coordinates,number of offices near
0,5e049e9de6eb90cad2d707ad,Fixya,30.0,2013,web,52cdef7c4bab8bd675297fec,8000000.0,San Mateo,USA,37.566879,-122.323895,"{'type': 'Point', 'coordinates': [-122.323895,...",2
1,5e049e9de6eb90cad2d707ae,GENWI,25.0,2010,web,52cdef7c4bab8bd675297f9e,7100000.0,Los Altos,USA,33.817100,-111.903500,"{'type': 'Point', 'coordinates': [-111.9035, 3...",1
2,5e049e9de6eb90cad2d707af,Social Gaming Network,100.0,2011,web,52cdef7c4bab8bd67529831a,17100000.0,Los Angeles,USA,37.446823,-122.161523,"{'type': 'Point', 'coordinates': [-122.161523,...",3
3,5e049e9de6eb90cad2d707b0,Skydeck,9.0,2012,web,52cdef7c4bab8bd675298509,4000000.0,Palo Alto,USA,37.564538,-122.325470,"{'type': 'Point', 'coordinates': [-122.3254705...",2
4,5e049e9de6eb90cad2d707b1,PeekYou,20.0,2012,web,52cdef7c4bab8bd675297f94,1830000.0,New York,USA,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ...",7
5,5e049e9de6eb90cad2d707b2,Magento,275.0,2010,web,52cdef7c4bab8bd675298608,22500000.0,Los Angeles,USA,34.052187,-118.243425,"{'type': 'Point', 'coordinates': [-118.243425,...",1
6,5e049e9de6eb90cad2d707b3,BrandYourself,22.0,2009,web,52cdef7d4bab8bd675298ce2,1500000.0,New York,USA,42.275263,-71.247620,"{'type': 'Point', 'coordinates': [-71.24762, 4...",1
7,5e049e9de6eb90cad2d707b4,Social Gaming Network,100.0,2011,web,52cdef7c4bab8bd67529831a,17100000.0,Beverly Hills,USA,34.069849,-118.405418,"{'type': 'Point', 'coordinates': [-118.4054182...",1
8,5e049e9de6eb90cad2d707b5,Titan Gaming,18.0,2010,web,52cdef7d4bab8bd675298dcc,1000000.0,Santa Monica,USA,53.544711,-113.515769,"{'type': 'Point', 'coordinates': [-113.515769,...",1
9,5e049e9de6eb90cad2d707b7,Inuvo,50.0,2009,web,52cdef7d4bab8bd67529980b,4200000.0,Clearwater,USA,27.913011,-82.702449,"{'type': 'Point', 'coordinates': [-82.702449, ...",2


In [ ]:
def getNearby(lat,lng,query,radius):
        CLIENT_ID = os.getenv("client_id")
        CLIENT_SECRET = os.getenv("client_secret")
        url = 'https://api.foursquare.com/v2/venues/explore'
        queryParams = dict(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            v='20181023', # DONDE ESTOY
            ll=f"{lat},{lng}", #QUE QUIERO
            query=query, # A QUE DISTANCIA
            radius=radius
        )
        res = requests.get(url, params=queryParams)
        return res.json()
  

In [57]:
for coord in geo:
    for place, value in places.items():
        request =(getNearby(coord[1],coord[0],value['api'],value['distance']))
        

In [75]:
request['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4ade0d75f964a520ae6b21e3',
   'name': 'Recyclart',
   'location': {'address': 'Ursulinenstraat 25 Rue des Ursulines',
    'lat': 50.84151395313035,
    'lng': 4.34791361217189,
    'labeledLatLngs': [{'label': 'display',
      'lat': 50.84151395313035,
      'lng': 4.34791361217189}],
    'distance': 199,
    'postalCode': '1000',
    'cc': 'BE',
    'city': 'Brussel',
    'state': 'Brussels Hoofdstedelijk Gewest',
    'country': 'België',
    'formattedAddress': ['Ursulinenstraat 25 Rue des Ursulines',
     '1000 Brussel',
     'België']},
   'categories': [{'id': '4bf58dd8d48988d116941735',
     'name': 'Bar',
     'pluralName': 'Bars',
     'shortName': 'Bar',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'grou